In [1]:
import pandas as pd
import os
import functions
import psycopg2

In [2]:
path_2 = '../data'

In [3]:
# define an explicit path variable for processed data (required for pgAdmin4 to locate system files)
path_3 = r"D:\School\LHL\capstone_project_data\LHL_Capstone_Project\data"

FLNRO_WMB data cleaning (data too large to clean with Pandas)
- only wind data available (no solar)

Steps:
- load into pgAdmin
- remove irrelevant columns
- rename relevant columns
- drop rows without windspeed values
- convert windspeed from km/h to m/s
- remove rows with values outside of reasonable range
- replace "-" with "_" in station_id column
- add to wind_data_final

In [4]:
# create FLNRO_WMB table in pgAdmin
functions.create_sql_table('FLNRO_WMB_data.csv')

Creating FLNRO_WMB_data in database


In [5]:
# add column names
functions.add_sql_columms('FLNRO_WMB_data.csv', path_3)

Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data
Adding columns to FLNRO_WMB_data


In [6]:
# add values
functions.add_values('FLNRO_WMB_data.csv', path_3)

Adding values to FLNRO_WMB_data


In [ ]:
# count nulls in each column
# """
# select key as column, count(*) as null_values
# from my_table t
# cross join jsonb_each_text(to_jsonb(t))
# where value is null
# group by key;
# """

'\nselect key as column, count(*) as null_values\nfrom my_table t\ncross join jsonb_each_text(to_jsonb(t))\nwhere value is null\ngroup by key;\n'

Output:
"column"	                "null_values"
"air_temp"	                47714451
"avg_wnd_dir_10m_pst10mts"	47683610
"avg_wnd_spd_10m_pst10mts"	48113491
"dwpt_temp"	                47739012
"precipitation"	            528426
"rel_hum"	                47636480
"relative_humidity"	        576502
"rnfl_amt_pst1hr"	        47636005
"rnfl_amt_pst24hrs"	        48040073
"snw_dpth"	                51832134
"temperature"	            916768
"wind_direction"	        677900
"wind_speed"	            567310

In [7]:
# drop everything except wind_speed, time, station_id, network_id
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

table_name = 'flnro_wmb_data.csv'.split('.csv')[0]
sql1 = f"""ALTER TABLE {table_name}
        ALTER COLUMN time
        TYPE timestamp
        USING time::timestamp without time zone;
"""
print(f"Converting 'time' column in {table_name} to timestamp")
cursor.execute(sql1)
con.commit()


Converting 'time' column in flnro_wmb_data to timestamp


In [8]:
# create flnro_wmb_wind table
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

sql2 = f"""
DROP TABLE IF EXISTS flnro_wmb_wind;
CREATE TABLE flnro_wmb_wind
(LIKE flnro_fern_wind);
"""
print(f"Creating flnro_wmb_wind table")
cursor.execute(sql2)
con.commit()

Creating flnro_wmb_wind table


In [9]:
# copy relevant values into flnro_wmb_wind
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

sql3 = f"""
    INSERT INTO flnro_wmb_wind(wind_speed, time, station_id, network_id)
    SELECT wind_speed, time, station_id, network_id 
    FROM flnro_wmb_data;
"""
print(f"Adding values from flnro_wmb_data to flnro_wmb_wind")

cursor.execute(sql3)
con.commit()
print(f"Successfully added values")

Adding values from flnro_wmb_data to flnro_wmb_wind
Successfully added values


In [10]:
# drop rows where wind_speed is null
# initial count of fnlro_wmb_wind: 53568449
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

sql = f"""
    DELETE FROM flnro_wmb_wind
    WHERE wind_speed IS NULL;
"""
print(f"Deleting rows from flnro_wmb_wind where wind_speed is null")

cursor.execute(sql)
con.commit()

print("Rows successfully deleted")

Deleting rows from flnro_wmb_wind where wind_speed is null
Rows successfully deleted


Row count after deleting: 53001139

In [11]:
# convert wind speed from km/h to m/s
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

sql = """UPDATE flnro_wmb_wind SET wind_speed = wind_speed / 3.6"""

print('Converting wind_speed to m/s')

cursor.execute(sql)
con.commit()

print('Column successfully converted')

Converting wind_speed to m/s
Column successfully converted


In [12]:
# remove values outside of reasonable range
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

sql = """DELETE FROM flnro_wmb_wind 
        WHERE wind_speed > 65 OR wind_speed < 0
"""

print('Deleting rows where wind_speed values are out of reasonable range')

cursor.execute(sql)
con.commit()

print('Rows successfully deleted')

Deleting rows where wind_speed values are out of reasonable range
Rows successfully deleted


In [13]:
# convert network_id from FLNRO-WMB to FLNRO_WMB
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

sql = """UPDATE flnro_wmb_wind SET network_id = REPLACE(network_id, 'FLNRO-WMB', 'FLNRO_WMB')"""

print('Converting FLNRO-WMB to FLNRO_WMB')

cursor.execute(sql)
con.commit()

print('Network ID successfully converted')

Converting FLNRO-WMB to FLNRO_WMB
Network ID successfully converted


In [15]:
# add values to wind_data
postgres_password = os.environ['POSTGRES_PASS']
con = psycopg2.connect(
    database='lhl_capstone_project',
    user='postgres',
    password=f'{postgres_password}',
    host='localhost',
    port='5432'
)
con.rollback()
cursor = con.cursor()

sql = """INSERT INTO wind_data(wind_speed, time, station_id, network_id)
        SELECT wind_speed, time, station_id, network_id
        FROM flnro_wmb_wind
"""

print('Copying flnro_wmb_wind values to wind_data')
cursor.execute(sql)
con.commit()
print('Successfully copied values')

Copying flnro_wmb_wind values to wind_data
Successfully copied values
